PART 1

In [2]:
from collections import  defaultdict

with open("Data/ES/train") as f:
    es = f.read().splitlines()

with open("Data/RU/train") as f:
    ru = f.read().splitlines()


In [3]:
def estimate_emission_params(train_data,k=1):

    word_sentiment_counts = defaultdict(lambda: defaultdict(int))
    sentiment_counts =  defaultdict(int)
    emission_params = {}
    
    #getting the count(y) and count(y --> x)
    for sentence in train_data:
        try:
            if(sentence!=""):
                x, label = sentence.split(" ")
        except:
            continue

        sentiment_counts[label]+=1

        word_sentiment_counts[label][x]+=1



    #calculating the emission parameters

    for key in word_sentiment_counts:
        for word in word_sentiment_counts[key]:
            emission_params[(word,key)] = word_sentiment_counts[key][word]/(sentiment_counts[key])
    return emission_params, sentiment_counts

PART 2

In [4]:
#Getting the data from ES 
# train_es = read_data('./Data/ES/train')

def estimate_emission_params_unk(x,y,k=1):
    if x=="#UNK":
        return k/(y+k)


    # count_y_to_x, count_y = 0, 0
    # for i in range(len(tags)):
    #     if tags[i] == y:
    #         count_y += 1
    #         if words[i] == x:
    #             count_y_to_x += 1
    # return (count_y_to_x) / (count_y + k)
    
    # word_sentiment_counts = defaultdict(lambda: defaultdict(int))
    # sentiment_counts = defaultdict(lambda: 1)
    # emission_params = {}
    
    # #getting the count(y) and count(y --> x)
    # for sentence in train_data:
    #     try:
    #         if(sentence!=""):
    #             x, label = sentence.split(" ")
    #     except:
    #         continue

    #     sentiment_counts[label]+=1

    #     word_sentiment_counts[label][x]+=1


    # #changing count(y->x) when x="#UNK"
    # for key in word_sentiment_counts:
    #     word_sentiment_counts[key]["#UNK#"]=k


    # #calculating the emission parameters

    # for key in word_sentiment_counts:
    #     for word in word_sentiment_counts[key]:
    #         emission_params[(word,key)] = word_sentiment_counts[key][word]/(sentiment_counts[key]+k)
    # return emission_params


In [22]:
#Getting the data from ES 
# train_es = read_data('./Data/ES/train')

def estimate_emission_params_unk(x,tag, y,k=1):
    if x=="#UNK":
        return k/(y[tag]+k)


    # count_y_to_x, count_y = 0, 0
    # for i in range(len(tags)):
    #     if tags[i] == y:
    #         count_y += 1
    #         if words[i] == x:
    #             count_y_to_x += 1
    # return (count_y_to_x) / (count_y + k)
    
    # word_sentiment_counts = defaultdict(lambda: defaultdict(int))
    # sentiment_counts = defaultdict(lambda: 1)
    # emission_params = {}
    
    # #getting the count(y) and count(y --> x)
    # for sentence in train_data:
    #     try:
    #         if(sentence!=""):
    #             x, label = sentence.split(" ")
    #     except:
    #         continue

    #     sentiment_counts[label]+=1

    #     word_sentiment_counts[label][x]+=1


    # #changing count(y->x) when x="#UNK"
    # for key in word_sentiment_counts:
    #     word_sentiment_counts[key]["#UNK#"]=k


    # #calculating the emission parameters

    # for key in word_sentiment_counts:
    #     for word in word_sentiment_counts[key]:
    #         emission_params[(word,key)] = word_sentiment_counts[key][word]/(sentiment_counts[key]+k)
    # return emission_params


In [19]:
# results of training using es and ru datasets for part 1
result_es = estimate_emission_params(es)
result_ru = estimate_emission_params(ru)
# # results of training using es and ru datasets for part 2
# result_es_unk = estimate_emission_params_unk(es)
# result_ru_unk = estimate_emission_params_unk(ru)

PART 3

In [20]:
with open("Data/ES/dev.in") as f:
    dev_in_es = f.read().splitlines()
with open("Data/ES/dev.out") as f:
    dev_out_es = f.read().splitlines()
with open("Data/RU/dev.in") as f:
    dev_in_ru = f.read().splitlines()
with open("Data/RU/dev.out") as f:
    test_out_ru = f.read().splitlines()

In [21]:
tags = ["B-neutral", "B-positive", "B-negative", "I-neutral", "I-positive", "I-negative", "O"]

result_es, count_y = estimate_emission_params(es)
y_optimal = defaultdict(str)
for word, sentiment in result_es:
    max1 = []
    for tag in tags:
        if tag == sentiment:
            max1.append((tag, result_es[(word, tag)]))
    y_optimal[word] = max(max1, key=lambda x: x[1])[0]

In [25]:
with open('./Data/ES/dev.in') as f:
    x = f.readlines()
with open('./EvalScript/dev.p1.out', 'w+') as f:
        
    for i in x:
        if i == '\n':
            f.write(i)
        else:
            word = i.strip()
            if word in y_optimal:
                f.write(word + ' ' + y_optimal[word] + '\n')
            else:
                # if the word is an unk, use the emission_unk function to find the y_max and write it into the file
                y_max_prob=0
                y_max=""
                for y in tags:
                    ep=estimate_emission_params_unk("#UNK",y,count_y)
                    if ep>y_max_prob:
                        y_max_prob=ep
                        y_max=y
                f.write(word +" "+y_max+"\n")
        

In [9]:
#FINDING THE MAXIMUM VLLAUE OF e(x|y)
tags = ["B-neutral", "B-positive", "B-negative", "I-neutral", "I-positive", "I-negative", "O"]

##########################################################
#updtaing the k value for the new strings not seen by the training data
# k=0
# for word in dev_in_es:
#     if word not in result_es_unk:
#         k += 1
##########################################################
#Stroing the probablities and tags into holders
em_prm,sentiment_cnt=estimate_emission_params(es)
holder=defaultdict(list)
holder2=defaultdict(list)
for i in em_prm:
    holder[i[0]].append(em_prm[i])   
    holder2[i[0]].append(i[1])   
##########################################################
# Getting the maximum value of the probabiliites and using 
# it to get the index to gewt the correct tag
y_star={}
for i in holder:
    index=holder[i].index(max(holder[i]))
    y_star[i]=holder2[i][index]
##########################################################
# writing y_star into dev.p1.out


test_file=open("./Data/ES/dev.in","r")
pred_output=open("dev.p1.out","w+")
# for i in y_star.keys():
#     print(i)
for l in dev_in_es:

    if l=="\n":
        pred_output.write(l)
    else:
        if l in y_star.keys() :
            pred_output.write(l+" "+y_star[l]+"\n")
        else:
            arr4=[]
            arr3=[]
            for i in tags:
                y=sentiment_cnt[i]
                z=estimate_emission_params_unk("#UNK",y)
                arr4.append(z)
                arr3.append(i)
  
            z=arr4.index(max(arr4))
            pred_output.write(l+" "+arr3[z].strip()+"\n")
            #function for update is UNIQUE
            
        

# with open('dev.p1.out', 'w', encoding='utf-8') as dev_out_file:
#     for predicted_labels in y_star:
#         dev_out_file.write(f"{predicted_labels}\t{y_star[predicted_labels]}\n")
#     dev_out_file.write('\n')

Estuvimos
hace
poco
mi
pareja
y
yo
comiendo
resultó
todo
muy
bien
,
tanto
la
el
…
nos
gustó
mucho
.
Por
poner
algún
pero
quizá
no
era
lo
"
ibérico
que
cabía
esperar
Bien
sabe
autor
del
blog
)
exquisita
diferente
creativo
agradable
Si
has
probado
sus
te
estas
perdiendo
algo
grande
!
En
resumen
bien-muy
correcto
profesional
02-12-
2012
elegimos
este
por
los
comentarios
ha
sido
una
Grandisima
Decepción
Salimos
encantadas
Ubicación
Comimos
atendidos
Los
en
estado
lamentable
etc
<->
resecas
sabemos
llenaron
con
trozos
de
esa
casi
caucho-imposible
comer
supuesto
ya
estoy
dando
voces
para
avisar
a
todas
mis
relaciones
Borja
Alagarmos
cena
hasta
las
tres
tomando
copas
Ofrecen
merluza
salsa
El
igual
Me
invitaron
restaurante
cual
desconocia
su
existencia
De
formas
decidimos
quedarnos
domingo
15
marzo
me
desplace
desde
galicia
unos
amigos
competir
maraton
bcn
Para
guiris
impresionante
atencion
aperitivo
es
ni
normal
bueno
un
recomendar
todos
Calidad
excelente
mesas
se
mostraron
atentos
momento
-


In [8]:
# Calculating the Precision, Recall and F score for the given data
def scores(labels, correct_labels):
    correct = 0
    total_predicted = 0
    for word in labels:
        if labels[word] == correct_labels[word]:
            correct += 1
        total_predicted += 1
        
    precision = correct / total


def F_score(recall,precision):
    v1=1/precision
    v2=1/recall
    f=2/(v1+v2)